# MDS DSCI 525 - Group 15 Milestone 1

**Author**: Lennon Lok Lam Au-Yeung, Ken Wang, Ty Andrews, Peng Zhang

## Step 0 Importing library

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Step 1 Downloading the data via API

Navigate to the location of your computer where you would like to download the files to.

In [3]:
#%cd ~/MDS/525_labs/figshareexp
%cd /Users/lennon/Downloads
## Change it to the location that you want to download your files to.

/Users/lennon/Downloads


In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"] 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 5.31 s, sys: 15.2 s, total: 20.5 s
Wall time: 7min 18s


Extracting the zip file.

In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.76 s, sys: 1.21 s, total: 8.97 s
Wall time: 9.16 s


In [7]:
%ls -ltr figsharerainfall

total 12048944
-rw-r--r--   1 lennon  staff  814041183 Mar 30 10:04 data.zip
-rw-r--r--   1 lennon  staff   95376895 Mar 30 10:04 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff   94960113 Mar 30 10:04 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff   82474546 Mar 30 10:04 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff  127613760 Mar 30 10:04 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff  232118894 Mar 30 10:04 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff  330360682 Mar 30 10:04 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff  254009247 Mar 30 10:04 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff  235661418 Mar 30 10:04 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff  294260911 Mar 30 10:04 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff  295768615 Mar 30 10:04 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 lennon  staff  328852

## Step 2 Combining data CSVs

Combine csv files into one file. Note that `observed_daily_rainfall_SYD.csv` has been manually removed as per the milestone 1 requirement.

In [8]:
%%time
# We are using a normal python way for merging the data 
# add extra column of "model"
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

CPU times: user 3min 18s, sys: 8.75 s, total: 3min 26s
Wall time: 3min 28s


Compare the time for combining CSVs on team member's local computers. See the following table for results.

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Lennon Lok Lam |  MacOS Ventura V13.2.1 |  16GB  | Apple M1 Pro  | Yes | 3min 25s |
| Ken            | Ubuntu 18.04 | 4GB + 8GB swap|Intel N4020 @ 1.10Ghz |Yes | 28min |
| Ty             |  Windows 11  |32GB | Intel 11th Gen i7  | Yes  | 13min 41s  |
| Peng           | MacOS Ventura V13.2.1 | 16GB | Apple M2 | Yes | 3min 8s  |

From the above table, we can see that machines with Apple ARM processors has a much quicker processing time than using Windows OS, even though it has more RAM and a recent generation of CPU regarding Ty's machine. This might be because of the efficiency of the Apple chip and its optimizaion of the MacOS system. All of our computers has SSD so we are not able to compare the results between SSD and HDD. 

Challenges faced Ken's machine:
The combining takes a lot memory. Since my laptop only has 4GB RAM, the notebook kernel crashed when it ran out of RAM. Then I added 8GB swap to the system and this time the combining code ran fine. The processing took 28 minutes and I think it's so slow mainly because it's using a lot swap.

## Step 3 Load combined CSV to memory and perform a simple EDA in Python

We have tried changing the `dtype` of our data and loading in chunks

In [9]:
df = pd.read_csv("figsharerainfall/combined_data.csv", index_col = 'time')
df2 = pd.read_csv("figsharerainfall/combined_data.csv", index_col = 'time',
                  dtype= {'lat_min':'float32','lat_max':'float32','lon_min':'float32','lon_max':'float32','rain (mm/day)':'float32'})

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB


In [11]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df2.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3498.20 MB
Memory usage with float32: 2248.84 MB


As we can see from the output above, using `float32` has a lower memory usage.

The following we tried loading the data and doing EDA in the normal way that we usually do.

In [12]:
%%time
df = pd.read_csv("figsharerainfall/combined_data.csv", index_col = 'time')
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 32.2 s, sys: 4.22 s, total: 36.4 s
Wall time: 37.6 s


This time we tried loading the data in chuncks and counting them in chunks

In [13]:
%%time
counts = pd.Series(dtype=int)
for chunk in  pd.read_csv("figsharerainfall/combined_data.csv",
                          chunksize=10_000_000, usecols=['model']):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int).sort_values(ascending = False))

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
dtype: int64
CPU times: user 17.6 s, sys: 1.68 s, total: 19.3 s
Wall time: 19.4 s


As we can see from the comparison above, doing EDA only with the columns we need and loading it in chunks have reduced the time required to complete EDA.

Compare the time for `value_counts` on team member's local computers. See the following table for results.

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Lennon|  MacOS Ventura V13.2.1 |  16GB  | Apple M1 Pro  | Yes | 19s|
| Ken            |  Ubuntu 18.04|4GB + 8GB swap| Intel N4040 @ 1.10GHz|Yes| 1m 30s |
| Ty             |  Windows 11  |32GB | Intel 11th Gen i7  | Yes  |53s |
| Peng           | MacOS Ventura V13.2.1 | 16GB | Apple M2 | Yes | 16s  |

Similar with the results of combining data, the both Apple machines are once again the faster than Ty's machine and Ken's machine is the slowest possibly due to a weaker processor and swap ram used.

## Step 4 Perform a simple EDA in R

To understand how Python and R can work together using Arrow we will load data in Python to an Arrow dataset then make it available to R for the EDA using rpy2_arrow. We chose to use the arrow based data transfer method between Python & R because it is the most efficient memory/storage solution for exchanging data between Python and R. Although exporting a snapshot of data to parquet format can be advantageous for modularity of analyses, this is a simple and inelegant method to solving the problem of sharing data between processes.

In [1]:
import os
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow 
from pyarrow import csv
import rpy2

os.environ['R_HOME'] = '/opt/miniconda3/envs/525_2023/lib/R'

# to use R cll magic
%load_ext rpy2.ipython

# keep this below load_ext command, bug makes it break if it is above, SOF issue: https://github.com/rpy2/rpy2/issues/874
import rpy2_arrow.pyarrow_rarrow as pyra

In [10]:
%%time
filepathcsv = "figsharerainfall/combined_data.csv"
filepathparquet = "figsharerainfall/combined_data.parquet"
df = ds.dataset(filepathcsv, format="csv")
result = df.scanner(columns=["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)","model"])
ds.write_dataset(result,filepathparquet,format = "parquet")

CPU times: user 22.6 s, sys: 1.7 s, total: 24.3 s
Wall time: 12.4 s


In [11]:
%%time
parquet_path = os.path.join("figsharerainfall", "combined_data.parquet") 

dataset = ds.dataset(parquet_path, format="parquet")
# Converting the `pyarrow dataset` to a `pyarrow table`
table = dataset.to_table()
# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

CPU times: user 3.01 s, sys: 1 s, total: 4.02 s
Wall time: 1.1 s


Check what the head and tail of our data looks like.

In [19]:
%%time
%%R -i r_table
start_time <- Sys.time()
suppressMessages({
  library(dplyr)
  library(arrow)
})
h <- head(r_table, 5) |>
    collect()
t <- tail(r_table, 5) |>
    collect()
print("Head: ")
print(h)
print("Tail: ")
print(t)
end_time <- Sys.time()
print(end_time - start_time)

[1] "Head: "
# A tibble: 5 × 7
  time                lat_min lat_max lon_min lon_max `rain (mm/day)` model     
  <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>     
1 1889-01-01 04:00:00   -35.4   -33.6    142.    143.        4.24e-13 MPI-ESM-1…
2 1889-01-02 04:00:00   -35.4   -33.6    142.    143.        4.22e-13 MPI-ESM-1…
3 1889-01-03 04:00:00   -35.4   -33.6    142.    143.        4.50e-13 MPI-ESM-1…
4 1889-01-04 04:00:00   -35.4   -33.6    142.    143.        4.25e-13 MPI-ESM-1…
5 1889-01-05 04:00:00   -35.4   -33.6    142.    143.        4.27e-13 MPI-ESM-1…
[1] "Tail: "
# A tibble: 5 × 7
  time                lat_min lat_max lon_min lon_max `rain (mm/day)` model     
  <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>     
1 2014-12-27 04:00:00   -30.2   -29.2    153.    154.            6.69 SAM0-UNIC…
2 2014-12-28 04:00:00   -30.2   -29.2    153.    154.            7.86 SAM0-UNIC…
3 2014-12-29 04:00:00   -30.2   -29.2    153.  

What is the average/min/max rain fall for each model across the whole dataset?

In [21]:
%%time
%%R -i r_table
start_time <- Sys.time()

result <- r_table |>
    group_by(model) |>
    summarise(
        mean_rain = mean(`rain (mm/day)`), # need back ticks to use column names with spaces
        min_rain = min(`rain (mm/day)`),
        max_rain = max(`rain (mm/day)`),
    ) |>
    arrange(desc(mean_rain)) |>
    collect()

print(result)
end_time <- Sys.time()
print(end_time - start_time)

# A tibble: 27 × 4
   model         mean_rain  min_rain max_rain
   <chr>             <dbl>     <dbl>    <dbl>
 1 INM-CM4-8          2.81  0            213.
 2 INM-CM5-0          2.67  0            184.
 3 CMCC-CM2-SR5       2.38 -4.08e-13     203.
 4 MIROC6             2.30  2.13e-32     227.
 5 CMCC-CM2-HR4       2.28  0            224.
 6 CMCC-ESM2          2.27 -7.87e-14     230.
 7 NorESM2-MM         2.23  0            264.
 8 NorESM2-LM         2.23  0            131.
 9 TaiESM1            2.22 -3.81e-12     201.
10 ACCESS-ESM1-5      2.22 -3.05e-18     202.
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
Time difference of 0.301497 secs
CPU times: user 1.81 s, sys: 23 ms, total: 1.84 s
Wall time: 312 ms


What's the most North/South coordinates?

In [22]:
%%time
%%R -i r_table
# using summarize check the distribution of rain (mm/day) by model
north_south_results <- r_table |>
    group_by(model) |>
    summarise(
        most_north = max(lat_max),
        most_south = min(lat_min),
    ) |>
    arrange(desc(most_north)) |>
    collect()

print(north_south_results)

# A tibble: 27 × 3
   model         most_north most_south
   <chr>              <dbl>      <dbl>
 1 BCC-ESM1           -27.9      -36.3
 2 CanESM5            -27.9      -36.3
 3 KIOST-ESM          -28.1      -35.6
 4 FGOALS-g3          -28.4      -36.5
 5 NorESM2-LM         -28.4      -36  
 6 INM-CM4-8          -28.5      -36  
 7 INM-CM5-0          -28.5      -36  
 8 ACCESS-CM2         -28.8      -36.2
 9 MPI-ESM1-2-HR      -29.0      -36.5
10 GFDL-CM4           -29        -36  
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 1.54 s, sys: 23.8 ms, total: 1.57 s
Wall time: 285 ms
